# Hackathon - January 2023



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data1 = pd.read_csv("Downloads/corona_tested_individuals_ver_006.english.csv")
data2 = pd.read_csv("Downloads/corona_tested_individuals_ver_0083.english.csv/corona_tested_individuals_ver_0083.english.csv")

/home/readic/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/readic/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Part 1 - Attempt to set up a LGBM model using the original paper's criteria, to replicate a baseline model

First step for the questions posed is to recreate as closely as possible the LGBM model 

###### Section 1 - recreate the sample dataset used by the paper

In [3]:
data1.info

<bound method DataFrame.info of          test_date cough fever sore_throat shortness_of_breath head_ache  \
0       2020-04-30     0     0           0                   0         0   
1       2020-04-30     1     0           0                   0         0   
2       2020-04-30     0     1           0                   0         0   
3       2020-04-30     1     0           0                   0         0   
4       2020-04-30     1     0           0                   0         0   
...            ...   ...   ...         ...                 ...       ...   
278843  2020-03-11     0     0           0                   0         0   
278844  2020-03-11     0     0           0                   0         0   
278845  2020-03-11     0     0           0                   0         0   
278846  2020-03-11     0     0           0                   0         0   
278847  2020-03-11     0     0           0                   0         0   

       corona_result age_60_and_above  gender         t

In [4]:
## Check for any missing values
data1.isna().sum()

test_date              0
cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
corona_result          0
age_60_and_above       0
gender                 0
test_indication        0
dtype: int64

In [5]:
data1_removeNoGender = data1.copy().query('gender != "None"')
print(data1_removeNoGender.shape)

data1_removeNoGenderOtherResult = data1_removeNoGender.copy().query('corona_result != "other"')
print(data1_removeNoGenderOtherResult.shape)

(259285, 10)
(255911, 10)


In [6]:
data1_originalPeriod = data1_removeNoGenderOtherResult.copy()
data1_originalPeriod = data1_originalPeriod[data1_originalPeriod['test_date'].isin(['2020-04-07',
       '2020-04-06', '2020-04-05', '2020-04-04', '2020-04-03',
       '2020-04-02', '2020-04-01', '2020-03-31', '2020-03-30',
       '2020-03-29', '2020-03-28', '2020-03-27', '2020-03-26',
       '2020-03-25', '2020-03-24', '2020-03-23', '2020-03-22'])]

In [7]:
data1_originalPeriod_tidy = data1_originalPeriod.copy()
data1_originalPeriod_tidy['test_indication'] = data1_originalPeriod_tidy['test_indication'].map({'Contact with confirmed': 1, 'Other': 0, 'Abroad': 0})
data1_originalPeriod_tidy['age_60_and_above'] = data1_originalPeriod_tidy['age_60_and_above'].map({'Yes': 1, 'No': 0})
data1_originalPeriod_tidy['gender'] = data1_originalPeriod_tidy['gender'].map({'male': 1, 'female': 0})
data1_originalPeriod_tidy['corona_result'] = data1_originalPeriod_tidy['corona_result'].map({'positive': 1, 'negative': 0})

data1_removeNoGenderOtherResultMissingSymptoms = data1_originalPeriod_tidy.copy().query('cough != "None"')
data1_removeNoGenderOtherResultMissingSymptoms = data1_removeNoGenderOtherResultMissingSymptoms.copy().query('fever != "None"')
data1_removeNoGenderOtherResultMissingSymptoms = data1_removeNoGenderOtherResultMissingSymptoms.copy().query('sore_throat != "None"')
data1_removeNoGenderOtherResultMissingSymptoms = data1_removeNoGenderOtherResultMissingSymptoms.copy().query('shortness_of_breath != "None"')
data1_removeNoGenderOtherResultMissingSymptoms = data1_removeNoGenderOtherResultMissingSymptoms.copy().query('head_ache != "None"')


In [8]:
print(data1_removeNoGenderOtherResultMissingSymptoms.dtypes)

data1_removeNoGenderOtherResultMissingSymptoms['cough'] = data1_removeNoGenderOtherResultMissingSymptoms['cough'].astype(int)
data1_removeNoGenderOtherResultMissingSymptoms['fever'] = data1_removeNoGenderOtherResultMissingSymptoms['fever'].astype(int)
data1_removeNoGenderOtherResultMissingSymptoms['sore_throat'] = data1_removeNoGenderOtherResultMissingSymptoms['sore_throat'].astype(int)
data1_removeNoGenderOtherResultMissingSymptoms['shortness_of_breath'] = data1_removeNoGenderOtherResultMissingSymptoms['shortness_of_breath'].astype(int)
data1_removeNoGenderOtherResultMissingSymptoms['head_ache'] = data1_removeNoGenderOtherResultMissingSymptoms['head_ache'].astype(int)
data1_removeNoGenderOtherResultMissingSymptoms['gender'] = data1_removeNoGenderOtherResultMissingSymptoms['gender'].astype(int)

print(data1_removeNoGenderOtherResultMissingSymptoms.dtypes)

test_date              object
cough                  object
fever                  object
sore_throat            object
shortness_of_breath    object
head_ache              object
corona_result           int64
age_60_and_above        int64
gender                  int64
test_indication         int64
dtype: object
test_date              object
cough                   int64
fever                   int64
sore_throat             int64
shortness_of_breath     int64
head_ache               int64
corona_result           int64
age_60_and_above        int64
gender                  int64
test_indication         int64
dtype: object


In [9]:
odt = data1_removeNoGenderOtherResultMissingSymptoms.copy().drop(['test_date'],axis = 1)
odt.info

<bound method DataFrame.info of         cough  fever  sore_throat  shortness_of_breath  head_ache  \
162404      0      0            0                    0          0   
162405      0      0            0                    0          0   
162406      0      0            0                    0          0   
162407      0      0            0                    0          0   
162408      0      0            0                    0          0   
...       ...    ...          ...                  ...        ...   
265116      1      1            0                    0          0   
265117      1      1            1                    0          1   
265118      1      0            0                    0          0   
265119      1      0            0                    0          0   
265120      1      0            0                    0          0   

        corona_result  age_60_and_above  gender  test_indication  
162404              0                 0       1                0  
16240

In [10]:
odt.isna().sum()

cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
corona_result          0
age_60_and_above       0
gender                 0
test_indication        0
dtype: int64

###### Section 2 - recreate the light GBM model

In [11]:
import lightgbm as ltb

In [12]:
## Create test/train splits - original paper used 4:1 ratio so 20% test set
from sklearn.model_selection import train_test_split

X = odt.copy().drop(['corona_result'],axis = 1)
y = odt.copy()['corona_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 2022, stratify = y)


In [13]:
originalModel = ltb.LGBMClassifier(
    num_leaves = 20
    ,min_data_in_leaf = 4
    ,feature_fraction =  0.2
    ,bagging_fraction =  0.8
    ,bagging_freq =  5
    ,learning_rate = 0.05
    ,verbose = 1
    #,num_boost_round=603
    #,early_stopping_rounds=0
    ,random_state = 2020
)

In [14]:
originalModel.fit(X_train, y_train, eval_metric = 'auc',eval_set = [(X_test, y_test)],early_stopping_rounds = 5)
#originalModel.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Number of positive: 6692, number of negative

/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, feature_fraction=0.2,
               learning_rate=0.05, min_data_in_leaf=4, num_leaves=20,
               random_state=2020, verbose=1)

In [15]:
y_preds = originalModel.predict(X_test)

In [16]:
from sklearn.metrics import roc_auc_score

rocAucScore2020 = roc_auc_score(y_test,y_preds)
print(rocAucScore2020)

0.5


In [17]:
## Rather than attempting to reverse engineer the model, load the model from text files available on the github!

loadModel1 = ltb.Booster(model_file = "Downloads/paperModel1.txt")
loadModel2 = ltb.Booster(model_file = "Downloads/paperModel2.txt")

In [18]:
print(loadModel1)
print(loadModel2)

In [19]:
y_preds_lm1 = loadModel1.predict(X_test)
y_preds_lm2 = loadModel2.predict(X_test[['cough','fever','test_indication','gender','age_60_and_above']])

In [20]:
print("Model 1 roc_auc score:")
print(roc_auc_score(y_test,y_preds_lm1))

print("Model 2 roc_auc score:")
print(roc_auc_score(y_test,y_preds_lm2))

Model 1 roc_auc score:
0.9218068553290598
Model 2 roc_auc score:
0.778974528515778


## Part 2 - apply the model to different periods 

In [21]:
# Part 1 - need to take the data preparation steps explored above and create a function to apply to other datasets

def prepareData(X):
    X = X.dropna()
    X = X.copy().query('gender != "None"')
    X = X.query('corona_result != "other"')
    X = X.query('corona_result != "Other"')
    X['test_indication'] = X['test_indication'].map({'Contact with confirmed': 1, 'Other': 0, 'Abroad': 0})
    X['age_60_and_above'] = X['age_60_and_above'].map({'Yes': 1, 'No': 0})
    X['gender'] = X['gender'].map({'male': 1, 'female': 0})
    X['corona_result'] = X['corona_result'].map({'positive': 1, 'negative': 0})
    X = X.copy().query('cough != "None"')
    X = X.copy().query('fever != "None"')
    X = X.copy().query('sore_throat != "None"')
    X = X.copy().query('shortness_of_breath != "None"')
    X = X.copy().query('head_ache != "None"')
    X['cough'] = X['cough'].astype(int)
    X['fever'] = X['fever'].astype(int)
    X['sore_throat'] = X['sore_throat'].astype(int)
    X['shortness_of_breath'] = X['shortness_of_breath'].astype(int)
    X['head_ache'] = X['head_ache'].astype(int)
    X['gender'] = X['gender'].astype(int)
    X = X.drop(['test_date'],axis = 1)
    return X

In [22]:
period2 = prepareData(data2)

/home/readic/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [23]:
period2.isna().sum()

cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
corona_result          0
age_60_and_above       0
gender                 0
test_indication        0
dtype: int64

In [24]:
X_p2 = period2.copy().drop(['corona_result'],axis = 1)
y_p2 = period2.copy()['corona_result']
print(X_p2.shape)
print(y_p2.shape)

(2151898, 8)
(2151898,)


In [25]:
# Make predictions using 2022 Autumn-Winter data

y_pred_p2 = loadModel1.predict(X_p2)
print("Model 1 late 2020 roc_auc score:")
print(roc_auc_score(y_p2,y_pred_p2))

y_preds_p2_lm2 = loadModel2.predict(X_p2[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 late 2020 roc_auc score:")
print(roc_auc_score(y_p2,y_preds_p2_lm2))

Model 1 late 2020 roc_auc score:
0.8126525522716164
Model 2 late 2020 roc_auc score:
0.6790117994089835


In [26]:
loadModel1_retrain = loadModel1
loadModel2_retrain = loadModel2

In [27]:
## Retrain model to period 2 dataset
X_train_p2, X_test_p2, y_train_p2, y_test_p2 = train_test_split(X_p2, y_p2, test_size = 0.2,random_state = 2022, stratify = y_p2)


In [28]:

from sklearn.metrics import classification_report

y_pred_p2_B = loadModel1.predict(X_test_p2)
print("Model 1 late 2020 roc_auc score:")
print(roc_auc_score(y_test_p2,y_pred_p2_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_p2_B_lm2 = loadModel2.predict(X_test_p2[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 late 2020 roc_auc score:")
print(roc_auc_score(y_test_p2,y_preds_p2_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_p2,y_train_p2)
y_preds_p2_retrain = loadModel1_retrain.predict(X_test_p2)
print("Model 1 late 2020, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p2,y_preds_p2_retrain))

loadModel2_retrain = loadModel2_retrain.refit(X_train_p2[['cough','fever','test_indication','gender','age_60_and_above']],y_train_p2)
y_preds_p2_retrain_lm2 = loadModel2_retrain.predict(X_test_p2[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 late 2020, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p2,y_preds_p2_retrain_lm2))

Model 1 late 2020 roc_auc score:
0.8139458747270674
Model 2 late 2020 roc_auc score:
0.6793035094716672


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112816 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 1721518, number of used features: 8
Model 1 late 2020, after retrain - roc_auc score:
0.814039379496979


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.081436 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 1721518, number of used features: 5
Model 2 late 2020, after retrain - roc_auc score:
0.7324063745761428


In [29]:
newLGBM_2020_B = ltb.LGBMClassifier(
    num_leaves = 20
    ,min_data_in_leaf = 4
    #,feature_fraction =  0.2
    ,bagging_fraction =  0.8
    ,bagging_freq =  5
    ,learning_rate = 0.05
    #,verbose = 1
    #,num_boost_round=603
    #,early_stopping_rounds=0
    ,random_state = 2020
)
newLGBM_2020_B.fit(X_train_p2,y_train_p2)
y_preds_p2_newModel = newLGBM_2020_B.predict(X_test_p2)
print("Clean model late 2020 data - roc_auc score:")
print(roc_auc_score(y_test_p2,y_preds_p2_newModel))

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
Clean model late 2020 data - roc_auc score:
0.7817996180843221


In [30]:
## March 2021 sample data - check for further model deterioration
data3 = pd.read_csv("Downloads/corona_tests_2021_mar.csv")
data4 = pd.read_csv("Downloads/corona_tests_2021_sep.csv")

/home/readic/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
print(data3.shape)
print(data4.shape)

(429880, 10)
(717431, 10)


In [32]:
#data3['corona_result'].value_counts()

In [33]:
period3 = prepareData(data3)
print(period3.isna().sum())
X_p3 = period3.copy().drop('corona_result',axis = 1)
y_p3 = period3.copy()['corona_result']

X_train_p3, X_test_p3, y_train_p3, y_test_p3 = train_test_split(X_p3, y_p3, test_size = 0.2,random_state = 2022, stratify = y_p3)

cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
corona_result          0
age_60_and_above       0
gender                 0
test_indication        0
dtype: int64


/tmp/ipykernel_15012/1638675748.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['test_indication'] = X['test_indication'].map({'Contact with confirmed': 1, 'Other': 0, 'Abroad': 0})
/tmp/ipykernel_15012/1638675748.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age_60_and_above'] = X['age_60_and_above'].map({'Yes': 1, 'No': 0})
/tmp/ipykernel_15012/1638675748.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [34]:
y_pred_p3_B = loadModel1.predict(X_test_p3)
print("Model 1 early 2021 roc_auc score:")
print(roc_auc_score(y_test_p3,y_pred_p3_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_p3_B_lm2 = loadModel2.predict(X_test_p3[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 early 2021 roc_auc score:")
print(roc_auc_score(y_test_p3,y_preds_p3_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_p3,y_train_p3)
y_preds_p3_retrain = loadModel1_retrain.predict(X_test_p3)
print("Model 1 early 2021, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p3,y_preds_p3_retrain))



Model 1 early 2021 roc_auc score:
0.8383312575374029
Model 2 early 2021 roc_auc score:
0.7005886639387529


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022205 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 91047, number of used features: 8
Model 1 early 2021, after retrain - roc_auc score:
0.8383113183375138


In [35]:
loadModel2_retrain = loadModel2_retrain.refit(X_train_p3[['cough','fever','test_indication','gender','age_60_and_above']],y_train_p3)
y_preds_p3_retrain_lm2 = loadModel2_retrain.predict(X_test_p3[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 early 2021, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p3,y_preds_p3_retrain_lm2))

/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.097530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 91047, number of used features: 5
Model 2 early 2021, after retrain - roc_auc score:
0.7630770086576584


In [36]:

period4 = prepareData(data4)
print(period4.isna().sum())
X_p4 = period4.copy().drop('corona_result',axis = 1)
y_p4 = period4.copy()['corona_result']

X_train_p4, X_test_p4, y_train_p4, y_test_p4 = train_test_split(X_p4, y_p4, test_size = 0.2,random_state = 2022, stratify = y_p4)

y_pred_p4_B = loadModel1.predict(X_test_p4)
print("Model 1 late 2021 roc_auc score:")
print(roc_auc_score(y_test_p4,y_pred_p4_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_p4_B_lm2 = loadModel2.predict(X_test_p4[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 late 2021 roc_auc score:")
print(roc_auc_score(y_test_p4,y_preds_p4_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_p4,y_train_p4)
y_preds_p4_retrain = loadModel1_retrain.predict(X_test_p4)
print("Model 1 late 2021, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p4,y_preds_p4_retrain))

loadModel2_retrain = loadModel2_retrain.refit(X_train_p4[['cough','fever','test_indication','gender','age_60_and_above']],y_train_p4)
y_preds_p4_retrain_lm2 = loadModel2_retrain.predict(X_test_p4[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 late 2021, after retrain - roc_auc score:")
print(roc_auc_score(y_test_p4,y_preds_p4_retrain_lm2))

/tmp/ipykernel_15012/1638675748.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['test_indication'] = X['test_indication'].map({'Contact with confirmed': 1, 'Other': 0, 'Abroad': 0})
/tmp/ipykernel_15012/1638675748.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['age_60_and_above'] = X['age_60_and_above'].map({'Yes': 1, 'No': 0})
/tmp/ipykernel_15012/1638675748.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
corona_result          0
age_60_and_above       0
gender                 0
test_indication        0
dtype: int64
Model 1 late 2021 roc_auc score:
0.6542129586036616
Model 2 late 2021 roc_auc score:
0.5817846624344593
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009846 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 298537, number of used features: 8


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


Model 1 late 2021, after retrain - roc_auc score:
0.6543238869008279
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 298537, number of used features: 5


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


Model 2 late 2021, after retrain - roc_auc score:
0.6093139732127104


In [37]:
print(X_train.shape)
print(X_train_p2.shape)
print(X_train_p3.shape)
print(X_train_p4.shape)

print(X_test.shape)
print(X_test_p2.shape)
print(X_test_p3.shape)
print(X_test_p4.shape)

(79191, 8)
(1721518, 8)
(91047, 8)
(298537, 8)
(19798, 8)
(430380, 8)
(22762, 8)
(74635, 8)


In [38]:
## Accuracy threshold testing
t005_p4_o8 = (y_pred_p4_B >= 0.05) * 1
t010_p4_o8 = (y_pred_p4_B >= 0.1) * 1
t025_p4_o8 = (y_pred_p4_B >= 0.25) * 1
t030_p4_o8 = (y_pred_p4_B >= 0.3) * 1
t040_p4_o8 = (y_pred_p4_B >= 0.4) * 1
t050_p4_o8 = (y_pred_p4_B >= 0.5) * 1

print("Threshold level 0.05, period 4")
print(classification_report(y_test_p4,t005_p4_o8))
print("Threshold level 0.1, period 4")
print(classification_report(y_test_p4,t010_p4_o8))
print("Threshold level 0.25, period 4")
print(classification_report(y_test_p4,t025_p4_o8))
print("Threshold level 0.3, period 4")
print(classification_report(y_test_p4,t030_p4_o8))
print("Threshold level 0.4, period 4")
print(classification_report(y_test_p4,t040_p4_o8))
print("Threshold level 0.5, period 4")
print(classification_report(y_test_p4,t050_p4_o8))

Threshold level 0.05, period 4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     67932
           1       0.09      1.00      0.16      6703

    accuracy                           0.09     74635
   macro avg       0.04      0.50      0.08     74635
weighted avg       0.01      0.09      0.01     74635

Threshold level 0.1, period 4


/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/pyth

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     67932
           1       0.09      1.00      0.16      6703

    accuracy                           0.09     74635
   macro avg       0.04      0.50      0.08     74635
weighted avg       0.01      0.09      0.01     74635

Threshold level 0.25, period 4
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     67932
           1       0.54      0.05      0.10      6703

    accuracy                           0.91     74635
   macro avg       0.73      0.52      0.53     74635
weighted avg       0.88      0.91      0.88     74635

Threshold level 0.3, period 4
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     67932
           1       0.59      0.01      0.03      6703

    accuracy                           0.91     74635
   macro avg       0.75      0.51      0.49     74635
weighted avg 

/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/readic/anaconda3/lib/pyth

In [39]:
## Part 3 - New LGBM Classifier on latest dataset
from sklearn.metrics import RocCurveDisplay

newLGBM = ltb.LGBMClassifier(
    random_state = 2020
    #,boosting_type = 'dart'
    ,num_leaves = 8
    ,max_depth = 8
    ,metric = 'auc'
    ,learning_rate = 0.1
)

newLGBM.fit(X_train_p4,y_train_p4)
y_preds_nm = newLGBM.predict(X_test_p4)
print(y_preds_nm)
print(roc_auc_score(y_test_p4,y_preds_nm))
print(classification_report(y_test_p4,y_preds_nm))

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(
    base_estimator = DecisionTreeClassifier(random_state = 2022)
    ,random_state = 2022
    ,learning_rate = 0.5
    ,n_estimators = 100
)
abc.fit(X_train_p4,y_train_p4)
y_pred_abc = abc.predict(X_test_p4)
print(roc_auc_score(y_test_p4,y_pred_abc))
print(classification_report(y_test_p4,y_pred_abc))



newLGBM = ltb.LGBMClassifier(
    random_state = 2020
    #,boosting_type = 'dart'
    ,num_leaves = 8
    ,max_depth = 8
    ,metric = 'auc'
    ,learning_rate = 0.1
)

newLGBM.fit(X_train_p4[['cough','fever','test_indication','gender','age_60_and_above']],y_train_p4)
y_preds_nm = newLGBM.predict(X_test_p4[['cough','fever','test_indication','gender','age_60_and_above']])
print(y_preds_nm)
print(roc_auc_score(y_test_p4,y_preds_nm))
print(classification_report(y_test_p4,y_preds_nm))

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(
    base_estimator = DecisionTreeClassifier(random_state = 2022)
    ,random_state = 2022
    ,learning_rate = 0.5
    ,n_estimators = 100
)
abc.fit(X_train_p4[['cough','fever','test_indication','gender','age_60_and_above']],y_train_p4)
y_pred_abc = abc.predict(X_test_p4[['cough','fever','test_indication','gender','age_60_and_above']])
print(roc_auc_score(y_test_p4,y_pred_abc))
print(classification_report(y_test_p4,y_pred_abc))




[0 0 0 ... 0 0 0]
0.5949495207838706
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     67932
           1       0.58      0.20      0.30      6703

    accuracy                           0.92     74635
   macro avg       0.76      0.59      0.63     74635
weighted avg       0.90      0.92      0.90     74635

0.5936897826207704
              precision    recall  f1-score   support

           0       0.93      0.99      0.95     67932
           1       0.58      0.20      0.30      6703

    accuracy                           0.92     74635
   macro avg       0.75      0.59      0.63     74635
weighted avg       0.89      0.92      0.90     74635

[0 0 0 ... 0 0 0]
0.5778807840055658
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     67932
           1       0.56      0.17      0.26      6703

    accuracy                           0.91     74635
   macro avg       0.74      0.58  

In [41]:
## Final scoring

#### Third wave
w3 = pd.read_csv("Downloads/data_processed/third_wave_alpha_processed2.csv")
#w3 = prepareData(wave3)
print(w3.isna().sum())
X_w3 = w3.copy().drop('corona_result',axis = 1)
y_w3 = w3.copy()['corona_result']

X_train_w3, X_test_w3, y_train_w3, y_test_w3 = train_test_split(X_w3, y_w3, test_size = 0.2,random_state = 2022, stratify = y_w3)

y_pred_w3_B = loadModel1.predict(X_test_w3)
print("Model 1 wave 3 roc_auc score:")
print(roc_auc_score(y_test_w3,y_pred_w3_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_w3_B_lm2 = loadModel2.predict(X_test_w3[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 3 roc_auc score:")
print(roc_auc_score(y_test_w3,y_preds_w3_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_w3,y_train_w3)
y_preds_w3_retrain = loadModel1_retrain.predict(X_test_w3)
print("Model 1 wave 3, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w3,y_preds_w3_retrain))

loadModel2_retrain = loadModel2_retrain.refit(X_train_w3[['cough','fever','test_indication','gender','age_60_and_above']],y_train_w3)
y_preds_w3_retrain_lm2 = loadModel2_retrain.predict(X_test_w3[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 3, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w3,y_preds_w3_retrain_lm2))

#### Foruth wave
w4 = pd.read_csv("Downloads/data_processed/fourth_wave_delta_processed2.csv")
#w4 = prepareData(wave4)
print(w4.isna().sum())
X_w4 = w4.copy().drop('corona_result',axis = 1)
y_w4 = w4.copy()['corona_result']

X_train_w4, X_test_w4, y_train_w4, y_test_w4 = train_test_split(X_w4, y_w4, test_size = 0.2,random_state = 2022, stratify = y_w4)

y_pred_w4_B = loadModel1.predict(X_test_w4)
print("Model 1 wave 4 roc_auc score:")
print(roc_auc_score(y_test_w4,y_pred_w4_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_w4_B_lm2 = loadModel2.predict(X_test_w4[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 3 roc_auc score:")
print(roc_auc_score(y_test_w4,y_preds_w4_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_w4,y_train_w4)
y_preds_w4_retrain = loadModel1_retrain.predict(X_test_w4)
print("Model 1 wave 4, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w4,y_preds_w4_retrain))

loadModel2_retrain = loadModel2_retrain.refit(X_train_w4[['cough','fever','test_indication','gender','age_60_and_above']],y_train_w4)
y_preds_w4_retrain_lm2 = loadModel2_retrain.predict(X_test_w4[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 4, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w4,y_preds_w4_retrain_lm2))


#### Fifth wave
w5 = pd.read_csv("Downloads/data_processed/fifth_wave_omicron_processed2.csv")
#w5 = prepareData(wave5)
print(w5.isna().sum())
X_w5 = w5.copy().drop('corona_result',axis = 1)
y_w5 = w5.copy()['corona_result']

X_train_w5, X_test_w5, y_train_w5, y_test_w5 = train_test_split(X_w5, y_w5, test_size = 0.2,random_state = 2022, stratify = y_w5)

y_pred_w5_B = loadModel1.predict(X_test_w5)
print("Model 1 wave 5 roc_auc score:")
print(roc_auc_score(y_test_w5,y_pred_w5_B))
#print(classification_report(y_test_p2,y_pred_p2_B))

y_preds_w5_B_lm2 = loadModel2.predict(X_test_w5[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 5 roc_auc score:")
print(roc_auc_score(y_test_w5,y_preds_w5_B_lm2))

loadModel1_retrain = loadModel1_retrain.refit(X_train_w5,y_train_w5)
y_preds_w5_retrain = loadModel1_retrain.predict(X_test_w5)
print("Model 1 wave 5, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w5,y_preds_w5_retrain))

loadModel2_retrain = loadModel2_retrain.refit(X_train_w5[['cough','fever','test_indication','gender','age_60_and_above']],y_train_w5)
y_preds_w5_retrain_lm2 = loadModel2_retrain.predict(X_test_w5[['cough','fever','test_indication','gender','age_60_and_above']])
print("Model 2 wave 5, after retrain - roc_auc score:")
print(roc_auc_score(y_test_w5,y_preds_w5_retrain_lm2))

cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
age_60_and_above       0
gender                 0
test_indication        0
corona_result          0
dtype: int64
Model 1 wave 3 roc_auc score:
0.7994211243501196
Model 2 wave 3 roc_auc score:
0.676489397389172


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 607110, number of used features: 8
Model 1 wave 3, after retrain - roc_auc score:
0.7983726766134447


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028839 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 607110, number of used features: 5
Model 2 wave 3, after retrain - roc_auc score:
0.7436740631842564
cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
age_60_and_above       0
gender                 0
test_indication        0
corona_result          0
dtype: int64
Model 1 wave 4 roc_auc score:
0.6378048474735774
Model 2 wave 3 roc_auc score:
0.5840622672200418


...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 368718, number of used features: 5
Model 2 wave 4, after retrain - roc_auc score:
0.6073177324113159
cough                  0
fever                  0
sore_throat            0
shortness_of_breath    0
head_ache              0
age_60_and_above       0
gender                 0
test_indication        0
corona_result          0
dtype: int64
Model 1 wave 5 roc_auc score:
0.4776838523671778
Model 2 wave 5 roc_auc score:
0.46617360807291075


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012679 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 503660, number of used features: 8
Model 1 wave 5, after retrain - roc_auc score:
0.499940322490221


/home/readic/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043287 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10
[LightGBM] [Info] Number of data points in the train set: 503660, number of used features: 5
Model 2 wave 5, after retrain - roc_auc score:
0.4723540407498081
